#### Notes

In [16]:
# ###############################################################################
# 1. Load excel
# 2. Find only tabs with "P"
# 3. Count how many diameters (curve tables) there are in Column A
#    a. Make that many duplicate tabs
#    b. Rename tabs to include Diameter
#    c. Delete the unnecessary tables that don't match the diameter for the tab'  
# 4. save to excel file
###############################################################################

#### Diameter Extraction Function

In [17]:
"""Returns list of all the trim diameters from cells A10-A21"""    
def extract_diameters(sh):
    list_of_diams = []    
    for row in sh.iter_rows(min_row=10, max_col=1, max_row=21):
        for cell in row:
            if cell.value != None:
                list_of_diams.append(cell.value)
        
    return(list_of_diams)

#### Reset Column A function

In [18]:
"""Sets only 1 diameter in diameter column"""
def reset_column_A(diameter, sh, diameters_list):
    for count, value in enumerate(diameters_list):
        cell_to_change = "{}{}".format('A',10+count)
        cell_to_change2 = "{}{}".format('B',10+count)
        
        if count == 0:
            sh[cell_to_change].value = diameter
            sh[cell_to_change2].value = str(diameter) + '-' + str(diameter)
        else:
            sh[cell_to_change].value = ''
            sh[cell_to_change2].value = ''   
            
    return

#### Get Diameter Cells

In [19]:
""" Returns list of all cells with trims/diameters in curve PSD"""   
def getDiameterCells(list_of_trims):
    from openpyxl.utils.cell import get_column_letter

    max_cols = 21 * len(list_of_trims)
    row = 7    
    first_col = 4

    # Assuming diameters are listed from largest to smallest, here we locate where curve data is for other trims.
    diameter_cols_number = sorted(list(range(first_col,max_cols,21)), reverse=True)
    
    # Convert col numbers to col letters for excel
    diameter_cols_str = [get_column_letter(x) for x in diameter_cols_number]
    
    # Concatenate column letter and row number to create cell coordinates for each trim
    diameter_cells = [col_ltr + row for col_ltr  in diameter_cols_str]

    return diameter_cells

#### Remove Curve Data Function

In [20]:
""" Deletes curve data not pertaining to current diameter for the tab"""    
def removeCurveData(diameter, sh, list_of_trims):
    from openpyxl.utils.cell import coordinate_from_string, column_index_from_string, get_column_letter

    first_col = 4
    max_cols = 21 * len(list_of_trims)
    row = 7    

    # Assuming diameters are listed from largest to smallest, here we locate where curve data is for other trims.
    diameter_cols_idx = sorted(list(range(first_col,max_cols,21)), reverse=True)
   
    for col_index in diameter_cols_idx:
        # print(f'col_index: {col_index} type: {type(col_index)}')
        # print(f'col_index: {col_index} type: {type(col_index)}')
        cell = get_column_letter(col_index) + str(row)
        
        # Check diameter values in row 7. IF diameter doesn't match, delete the following 21 columns
        # print("Before Delete - Cell: {} contains {}".format(cell,sh[cell].value))
        if sh[cell].value != diameter:    
            sh.delete_cols(col_index, 21)
            # print("After Delete - Cell: {} contains {}".format(cell,sh[cell].value))
        # IF the diameter matches, keep
        else:
            continue

    return

#### Creates PSD containing only std models

In [21]:
import os
from openpyxl import load_workbook
import shutil

myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\12 NBS Curve PSD Separation\12.01 Original Files"
myFile = "GXS Curve_Conexus_V2.xlsx"
original = os.path.join(myDir, myFile)

filename, file_ext = myFile.split(os.extsep)

targetDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\12 NBS Curve PSD Separation\12.02 Output Files"
std_target = os.path.join(targetDir, filename + " - std models." + file_ext)
shutil.copyfile(original, std_target) # Create copy to leave original untouched

wb_std = load_workbook(std_target, data_only=True)

In [22]:
trim_count = 0

for sheet in wb_std.worksheets:
    if '-E' in sheet.title:
        wb_std.remove(sheet)    

    # If tab with "P" is found at end of title, expand the trims found there.
    elif sheet.title[-1] == 'P':
        
        # Extract all trims from the current sheet
        diameters_list = extract_diameters(sheet)
        # print(f"Diameters in sheet {sheet.title} are here: {diameters_list}")
       
        for eachDiameter in diameters_list:    
            trim_count += 1    

            # Add as many tabs as needed per diameter and rename to reflect diameter
            newTabName = sheet.title + '-' + str(eachDiameter) + '_Std'
            tab_to_copy = wb_std[sheet.title]  
            wb_std.copy_worksheet(tab_to_copy).title = newTabName
            
            # Remove all additional diameters in column A
            reset_column_A(eachDiameter, wb_std[newTabName], diameters_list)
            
            # Delete data that doesn't belong in the tab
            removeCurveData(eachDiameter, wb_std[newTabName], diameters_list)
        
        # This removes the original tab 
        wb_std.remove(sheet)    

print(f"Total Amount of Trims Found: {trim_count}")    
    
wb_std._sheets.sort(key=lambda ws: ws.title) # This sorts the new excel tabs by name.
wb_std.save(std_target)

#### Creates PSD containing only e models

In [69]:

e_target = os.path.join(targetDir, filename + " - e models." + file_ext)
shutil.copyfile(original, e_target) # Create copy to leave original untouched

wb_e = load_workbook(e_target, data_only=True)

In [70]:

for sheet in wb_e.worksheets:
    # If tab with "P" is found at end of title, remove from PSD
    if sheet.title[-1] == 'P':
        wb_e.remove(sheet)    
        
# wb_e._sheets.sort(key=lambda ws: ws.title) # This sorts the new excel tabs by name.
wb_e.save(e_target)